In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
compound_x_information = pd.read_csv(
    "v2/v20.meta.per_compound.txt", sep="\t", index_col=1
)

assert not compound_x_information.index.has_duplicates

assert kraft.is_sorted(compound_x_information["master_cpd_id"])

compound_x_information.index.name = "Compound"

compound_x_information

In [ ]:
target_gene_x_compound = kraft.binarize(
    compound_x_information["gene_symbol_of_protein_target"]
)

target_gene_x_compound.index.name = "Target Gene"

target_gene_x_compound.to_csv("target_gene_x_compound.tsv", sep="\t")

target_gene_x_compound

In [ ]:
target_activity_x_compound = kraft.binarize(
    compound_x_information["target_or_activity_of_compound"]
)

target_activity_x_compound.index.name = "Target Activity"

target_activity_x_compound.to_csv("target_activity_x_compound.tsv", sep="\t")

target_activity_x_compound

In [ ]:
cell_line_x_information = pd.read_csv(
    "v2/v20.meta.per_cell_line.txt", sep="\t", index_col=1
)

assert not cell_line_x_information.index.has_duplicates

assert kraft.is_sorted(cell_line_x_information["master_ccl_id"])

cell_line_x_information.index.name = "Cell Line"

cell_line_x_information

In [ ]:
viability_after_exposure = pd.read_csv("new-abs-auc-with-qc.txt", sep="\t")

viability_after_exposure

In [ ]:
ctrp_2_compound_x_cell_line = np.full(
    (compound_x_information.shape[0], cell_line_x_information.shape[0]), np.nan
)

compound_id_i = {
    id_: i for i, id_ in enumerate(compound_x_information["master_cpd_id"])
}

cell_line_id_i = {
    id_: i for i, id_ in enumerate(cell_line_x_information["master_ccl_id"])
}

for compound_id, cell_line_id, auc in viability_after_exposure[
    ["master_cpd_id", "master_ccl_id", "area_under_curve"]
].values:

    ctrp_2_compound_x_cell_line[
        compound_id_i[compound_id], cell_line_id_i[cell_line_id]
    ] = auc

ctrp_2_compound_x_cell_line = pd.DataFrame(
    ctrp_2_compound_x_cell_line,
    index=compound_x_information.index,
    columns=cell_line_x_information.index,
)

ctrp_2_compound_x_cell_line.dropna(axis=1, how="all", inplace=True)

ctrp_2_compound_x_cell_line.to_csv("ctrp_2.compound_x_cell_line.tsv", sep="\t")

ctrp_2_compound_x_cell_line

In [ ]:
ctrp_2_compound_x_cell_line_no_combination = ctrp_2_compound_x_cell_line.loc[
    ~ctrp_2_compound_x_cell_line.index.str.contains(":")
]

ctrp_2_compound_x_cell_line_no_combination.to_csv(
    "ctrp_2.compound_x_cell_line.no_combination.tsv", sep="\t"
)

ctrp_2_compound_x_cell_line_no_combination